In [1]:
import pandas as pd
import numpy as np

def tratar_ids_duplicados(df, coluna_id='ID_Registro'):
    """
    Verifica IDs duplicados e trata conforme as regras:
    - Se dados duplicados: exclui linha
    - Se dados diferentes: gera novo ID inteiro único
    """
    
    # Fazer cópia do dataframe para não modificar o original
    df_tratado = df.copy()
    
    # Identificar IDs duplicados
    ids_duplicados = df_tratado[df_tratado.duplicated(subset=[coluna_id], keep=False)]
    
    if ids_duplicados.empty:
        print("✅ Nenhum ID duplicado encontrado")
        return df_tratado
    
    print(f"⚠️  Encontrados {len(ids_duplicados)} registros com IDs duplicados")
    
    # Encontrar o maior ID atual para gerar novos IDs acima dele
    max_id = df_tratado[coluna_id].max()
    novos_ids_gerados = 0
    linhas_excluidas = 0
    
    # Lista de todos os IDs existentes (para garantir unicidade)
    ids_existentes = set(df_tratado[coluna_id].tolist())
    
    # Processar cada grupo de IDs duplicados
    for id_valor, grupo in ids_duplicados.groupby(coluna_id):
        if len(grupo) == 1:
            continue  # Apenas um registro, não precisa tratar
            
        print(f"\nProcessando ID {id_valor}: {len(grupo)} registros")
        
        # Verificar se todos os registros são completamente duplicados
        colunas_verificar = [col for col in grupo.columns if col != coluna_id]
        duplicados_completos = grupo.duplicated(subset=colunas_verificar, keep='first')
        
        # Identificar índices para exclusão e modificação
        indices_para_excluir = []
        indices_para_modificar = []
        
        for idx, is_duplicado in duplicados_completos.items():
            if is_duplicado:
                indices_para_excluir.append(idx)
            else:
                indices_para_modificar.append(idx)
        
        # Excluir registros duplicados (mantém apenas o primeiro de cada grupo duplicado)
        if indices_para_excluir:
            df_tratado = df_tratado.drop(indices_para_excluir)
            linhas_excluidas += len(indices_para_excluir)
            print(f"   ➖ Excluídas {len(indices_para_excluir)} linhas duplicadas")
        
        # Gerar novos IDs para registros com dados diferentes
        if indices_para_modificar:
            # Pular o primeiro registro (mantém o ID original)
            indices_para_modificar = indices_para_modificar[1:] if indices_para_modificar else []
            
            for idx in indices_para_modificar:
                # Gerar novo ID inteiro único
                novo_id = max_id + 1
                while novo_id in ids_existentes:
                    novo_id += 1
                
                # Atualizar o ID
                df_tratado.loc[idx, coluna_id] = novo_id
                ids_existentes.add(novo_id)
                max_id = max(max_id, novo_id)
                novos_ids_gerados += 1
                
                print(f"   🔄 Linha {idx}: ID modificado de {id_valor} para {novo_id}")
    
    # Relatório final
    print(f"\n{'='*60}")
    print("RELATÓRIO DO PROCESSAMENTO:")
    print(f"✅ Linhas excluídas (duplicados): {linhas_excluidas}")
    print(f"🆔 Novos IDs gerados: {novos_ids_gerados}")
    print(f"📊 Total de registros final: {len(df_tratado)}")
    
    # Verificar se ainda há duplicados após o tratamento
    duplicados_finais = df_tratado[df_tratado.duplicated(subset=[coluna_id], keep=False)]
    if duplicados_finais.empty:
        print("✅ Verificação final: Nenhum ID duplicado restante")
    else:
        print(f"⚠️  ATENÇÃO: Ainda existem {len(duplicados_finais)} IDs duplicados")
    
    return df_tratado

def verificar_duplicados_detalhado(df, coluna_id='ID_Registro'):
    """
    Função para verificar detalhadamente os IDs duplicados
    """
    print("🔍 VERIFICAÇÃO DETALHADA DE IDs DUPLICADOS")
    print("=" * 50)
    
    # Identificar IDs duplicados
    ids_duplicados = df[df.duplicated(subset=[coluna_id], keep=False)]
    
    if ids_duplicados.empty:
        print("Nenhum ID duplicado encontrado")
        return
    
    # Contar por ID
    contagem = ids_duplicados[coluna_id].value_counts()
    
    for id_valor, quantidade in contagem.items():
        registros = df[df[coluna_id] == id_valor]
        
        print(f"\nID: {id_valor} - {quantidade} registros")
        
        # Verificar se são completamente duplicados
        colunas_dados = [col for col in registros.columns if col != coluna_id]
        duplicados_exatos = registros.duplicated(subset=colunas_dados).all()
        
        if duplicados_exatos:
            print("   Status: DUPLICADOS COMPLETOS (dados idênticos)")
        else:
            print("   Status: DADOS DIFERENTES")
            # Mostrar colunas com diferenças
            colunas_diff = registros.columns[registros.nunique() > 1]
            print(f"   Colunas com diferenças: {list(colunas_diff)}")

# Exemplo de uso completo
def exemplo_completo():
    # Criar dados de exemplo para teste
    dados_exemplo = {
        'ID_Registro': [1, 2, 3, 1, 2, 4, 5, 5, 5, 6, 7],
        'Tipo_Eletronico': ['Celular', 'Notebook', 'Tablet', 'Celular', 'Monitor', 
                           'TV', 'Impressora', 'Impressora', 'Scanner', 'Mouse', 'Teclado'],
        'Ano_Fabricacao': [2020, 2021, 2022, 2020, 2019, 
                          2020, 2021, 2021, 2022, 2023, 2022],
        'Origem': ['Residencial', 'Comercial', 'Residencial', 'Residencial', 'Comercial',
                  'Residencial', 'Comercial', 'Comercial', 'Industrial', 'Residencial', 'Comercial'],
        'Peso_kg': [0.2, 2.5, 0.6, 0.2, 5.0, 8.0, 4.5, 4.5, 3.2, 0.1, 0.3]
    }
    
    df = pd.DataFrame(dados_exemplo)
    
    print("📋 DATAFRAME ORIGINAL:")
    print(df)
    print(f"\n📊 Total de registros: {len(df)}")
    
    # Verificar duplicados detalhadamente
    print("\n" + "="*60)
    verificar_duplicados_detalhado(df, 'ID_Registro')
    
    # Tratar duplicados
    print("\n" + "="*60)
    print("🔄 PROCESSANDO DUPLICADOS...")
    df_tratado = tratar_ids_duplicados(df, 'ID_Registro')
    
    # Mostrar resultado
    print("\n" + "="*60)
    print("📋 DATAFRAME TRATADO:")
    print(df_tratado)
    print(f"\n📊 Total de registros após tratamento: {len(df_tratado)}")
    
    return df_tratado

# Para usar com seus dados reais:
# df = pd.read_csv('seu_arquivo.csv')  # ou seu dataframe
# verificar_duplicados_detalhado(df, 'ID_Registro')
# df_tratado = tratar_ids_duplicados(df, 'ID_Registro')

# Executar exemplo
if __name__ == "__main__":
    df_resultado = exemplo_completo()

📋 DATAFRAME ORIGINAL:
    ID_Registro Tipo_Eletronico  Ano_Fabricacao       Origem  Peso_kg
0             1         Celular            2020  Residencial      0.2
1             2        Notebook            2021    Comercial      2.5
2             3          Tablet            2022  Residencial      0.6
3             1         Celular            2020  Residencial      0.2
4             2         Monitor            2019    Comercial      5.0
5             4              TV            2020  Residencial      8.0
6             5      Impressora            2021    Comercial      4.5
7             5      Impressora            2021    Comercial      4.5
8             5         Scanner            2022   Industrial      3.2
9             6           Mouse            2023  Residencial      0.1
10            7         Teclado            2022    Comercial      0.3

📊 Total de registros: 11

🔍 VERIFICAÇÃO DETALHADA DE IDs DUPLICADOS

ID: 5 - 3 registros
   Status: DADOS DIFERENTES
   Colunas com difer

### 🛠️ Propostas

1. **Pontos de Coleta Segmentados**
   * Centros urbanos: coleta de computadores, celulares e impressoras.
   * Municípios rurais: coleta de baterias, TVs e geladeiras.
   * Rotas logísticas otimizadas para transporte seguro de equipamentos pesados e tóxicos
2. **Campanhas Integradas de Educação Ambiental**
    * Associadas à localização de pontos de coleta.
    * Foco em conscientização sobre riscos de metais pesados e destinos corretos.
3. **Políticas de Incentivo e Responsabilidade do Fabricante**
    * Subsídios ou créditos para órgãos governamentais e empresas privadas que realizem descarte formal.
    * Parcerias com cooperativas e recicladores locais.
    * Adaptação de modelos de EPR inspirados em países como Alemanha, Japão e Coreia do Sul, responsabilizando fabricantes pelo destino final seguro de seus produtos
4. **Monitoramento Dinâmico**
   * Dashboards integrando origem, tipo, peso, destino e risco.
   * Identificação de municípios prioritários para intervenção rápida.
   * Avaliação contínua do impacto das campanhas e ajustes de rotas logísticas.
5. **Planejamento Regional Diferenciado**
    * Estratégias específicas para grandes centros urbanos, municípios médios e rurais.
    * Foco em redução de descarte informal em áreas críticas, sem negligenciar municípios menores que apresentam boas práticas.

O estudo evidencia que a gestão do lixo eletrônico na Amazônia deve ser multidimensional, considerando simultaneamente tipo de eletrônico, origem do resíduo, peso, risco tóxico, educação ambiental e infraestrutura logística. A integração desses fatores permite identificar padrões de risco, alocar recursos de forma estratégica e aumentar a taxa de descarte formal. A proposta de pontos de coleta segmentados, associada a campanhas educativas e monitoramento dinâmico, constitui uma abordagem prática e realista para reduzir impactos ambientais e proteger a saúde das populações amazônicas. O relatório reforça que políticas públicas isoladas não são suficientes; apenas uma abordagem integrada e orientada por dados pode transformar informação em ação eficaz, contribuindo para a preservação ambiental e para o cumprimento das metas de sustentabilidade na região amazônica.